# Method Stability

In this notebook we evaluate 2 modes of stability of our method.

1. Given that our neural network to find facial landmarks has a mean error of around 4mm, how much impact does this have on the location of Kocher's Point?
2. Given that Kocher's point moves around a bit, based on the landmarks, how much does this affect the location of the Target Point?

We assess this by calculating the location of KP and TP using the manually placed landmarks, and then moving the landmarks around a bit to see how much the location of KP and TP changes.
Based on earlier results, we know that the predicted landmarks have a mean error of 4mm, so we will go a small step further and move the landmarks around a bit more than that, to see how much the location of KP and TP changes.

In [ ]:
from pathlib import Path
from random import choice

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.stats import spearmanr

from evdplanner.cli import set_verbosity
from evdplanner.generation import measure_kocher
from evdplanner.geometry import Mesh
from evdplanner.linalg import Vec3
from evdplanner.markups import MarkupManager
from evdplanner.rendering import find_target, Ray, IntersectionSort

skin_mesh_file = "mesh_skin.stl"
ventricles_mesh_file = "mesh_ventricles.stl"

manual_landmarks_file = "landmarks_skin.mrk.json"

evd_file = "EVD.mrk.json"

samples_dir = Path(r"S:\E_ResearchData\evdplanner\Samples")
test_dir = Path(r"S:\E_ResearchData\evdplanner\Test")

scores_file = Path(r"S:\E_ResearchData\evdplanner\MajorityVoting.csv")

set_verbosity(0)

We'll use all patients in the dataset for this evaluation.

In [ ]:
patients = [x.resolve() for x in test_dir.iterdir() if x.is_dir()]
patients += [x.resolve() for x in samples_dir.iterdir() if x.is_dir()]
print(f"Found {len(patients)} patients")

In the paper we performed 1000 randomizations for statistical significance.
Now that we've got a bit more time after the resubmission, we can also check with more runs.

In [ ]:
records = []

check_radially = True
radius = 1.5
objective_distance_weight = 0.75
thickness_threshold = 10.0
depth_threshold = 80.0

n_tests = 5000  # 1000 for the resubmission
wiggles_per_patient = 1  # 10 for the resubmission
n_patients = len(patients)
max_error = 10.0

while len(records) < n_tests:
    patient = choice(patients)
    print(f"Processing patient {patient.name} ({len(records) + 1}/{n_tests})...")

    try:
        skin_mesh = Mesh.load(str(patient / skin_mesh_file))
        ventricles_mesh = Mesh.load(str(patient / ventricles_mesh_file))
        gt_landmarks = MarkupManager.load(patient / manual_landmarks_file)
    except:
        print(f"Skin Mesh or landmarks not found for patient {patient.name}.")
        continue

    gt_nasion = Vec3(*gt_landmarks.find_fiducial("Nasion").position)
    gt_left_ear = Vec3(*gt_landmarks.find_fiducial("Pre-Auricle Left").position)
    gt_right_ear = Vec3(*gt_landmarks.find_fiducial("Pre-Auricle Right").position)

    print("Measuring GT Kocher's points...")
    gt_left_kp, gt_right_kp = measure_kocher(
        mesh=skin_mesh,
        nasion=gt_nasion,
        left_ear=gt_left_ear,
        right_ear=gt_right_ear,
    )

    print("Finding GT target points...")
    gt_left_tp, _ = find_target(
        ventricles_mesh,
        gt_left_kp,
        check_radially=check_radially,
        radius=radius,
        objective_distance_weight=objective_distance_weight,
        thickness_threshold=thickness_threshold,
        depth_threshold=depth_threshold,
    )
    gt_right_tp, _ = find_target(
        ventricles_mesh,
        gt_right_kp,
        check_radially=check_radially,
        radius=radius,
        objective_distance_weight=objective_distance_weight,
        thickness_threshold=thickness_threshold,
        depth_threshold=depth_threshold,
    )

    for wiggle in range(wiggles_per_patient):
        print(f"Wiggling {wiggle + 1}/{wiggles_per_patient}...")

        left_record = {
            "Patient": patient.name,
            "Side": "Left",
        }
        right_record = {
            "Patient": patient.name,
            "Side": "Right",
        }

        wiggle_nasion = gt_nasion + Vec3(
            np.random.uniform(-max_error, max_error),
            np.random.uniform(-max_error, max_error),
            np.random.uniform(-max_error, max_error),
        )
        wiggle_left_ear = gt_left_ear + Vec3(
            np.random.uniform(-max_error, max_error),
            np.random.uniform(-max_error, max_error),
            np.random.uniform(-max_error, max_error),
        )
        wiggle_right_ear = gt_right_ear + Vec3(
            np.random.uniform(-max_error, max_error),
            np.random.uniform(-max_error, max_error),
            np.random.uniform(-max_error, max_error),
        )

        # Project the wiggled points to the skin mesh
        wiggle_nasion = skin_mesh.intersect(
            ray=Ray(skin_mesh.origin, (wiggle_nasion - skin_mesh.origin).unit_vector),
            sorting=IntersectionSort.Farthest,
        )
        wiggle_left_ear = skin_mesh.intersect(
            ray=Ray(skin_mesh.origin, (wiggle_left_ear - skin_mesh.origin).unit_vector),
            sorting=IntersectionSort.Farthest,
        )
        wiggle_right_ear = skin_mesh.intersect(
            ray=Ray(skin_mesh.origin, (wiggle_right_ear - skin_mesh.origin).unit_vector),
            sorting=IntersectionSort.Farthest,
        )

        if wiggle_nasion is None or wiggle_left_ear is None or wiggle_right_ear is None:
            print(f"Intersection failed for patient {patient.name}.")
            continue

        wiggle_nasion = wiggle_nasion.position
        wiggle_left_ear = wiggle_left_ear.position
        wiggle_right_ear = wiggle_right_ear.position

        left_record["N Error"] = (wiggle_nasion - gt_nasion).length
        right_record["N Error"] = (wiggle_nasion - gt_nasion).length
        left_record["LPA Error"] = (wiggle_left_ear - gt_left_ear).length
        right_record["LPA Error"] = (wiggle_left_ear - gt_left_ear).length
        left_record["RPA Error"] = (wiggle_right_ear - gt_right_ear).length
        right_record["RPA Error"] = (wiggle_right_ear - gt_right_ear).length

        for lm in ["N Error", "LPA Error", "RPA Error"]:
            print(f"Left  {lm} = {left_record[lm]} mm")
            print(f"right {lm} = {right_record[lm]} mm")

        print("Measuring predicted Kocher's points...")
        # Measure Kocher's points using the wiggled landmarks
        predicted_left_kp, predicted_right_kp = measure_kocher(
            mesh=skin_mesh,
            nasion=wiggle_nasion,
            left_ear=wiggle_left_ear,
            right_ear=wiggle_right_ear,
        )

        left_record[r"$K$ Difference"] = (predicted_left_kp - gt_left_kp).length
        right_record[r"$K$ Difference"] = (predicted_right_kp - gt_right_kp).length

        print(f"Left  K Difference = {left_record[r'$K$ Difference']} mm")
        print(f"Right K Difference = {right_record[r'$K$ Difference']} mm")

        print("Finding target points...")
        # Find target points using the predicted Kocher's points
        predicted_left_tp, _ = find_target(
            ventricles_mesh,
            predicted_left_kp,
            check_radially=check_radially,
            radius=radius,
            objective_distance_weight=objective_distance_weight,
            thickness_threshold=thickness_threshold,
            depth_threshold=depth_threshold,
        )
        predicted_right_tp, _ = find_target(
            ventricles_mesh,
            predicted_right_kp,
            check_radially=check_radially,
            radius=radius,
            objective_distance_weight=objective_distance_weight,
            thickness_threshold=thickness_threshold,
            depth_threshold=depth_threshold,
        )

        left_record[r"$T$ Difference"] = (predicted_left_tp - gt_left_tp).length
        right_record[r"$T$ Difference"] = (predicted_right_tp - gt_right_tp).length
        left_record["Distance"] = (predicted_left_tp - predicted_left_kp).length
        right_record["Distance"] = (predicted_right_tp - predicted_right_kp).length

        print(f"Left  T Difference = {left_record[r'$T$ Difference']} mm")
        print(f"Right T Difference = {right_record[r'$T$ Difference']} mm")

        records.extend([left_record, right_record])

In [ ]:
df = pd.DataFrame.from_records(records)
df.head()

In [ ]:
scores = pd.read_csv(scores_file)
scores = scores.rename(columns={"PatientID": "Patient", "Score": "Kakarla"})
df = df.merge(scores, on=["Patient", "Side"], how="left")
df.head()

In [ ]:
p = sns.pairplot(
    data=df,
    hue="Side",
    markers=["o", "X"],
    diag_kind=None,
    x_vars=[
        "N Error",
        "LPA Error",
        "RPA Error",
    ],
    y_vars=[
        r"$K$ Difference",
    ],
)

p.figure.suptitle(r"$K$ deviation versus landmark error", y=1.1)

plt.show()

In [ ]:
pairs = [
    ("N Error", r"$K$ Difference"),
    ("LPA Error", r"$K$ Difference"),
    ("RPA Error", r"$K$ Difference"),
    (r"$K$ Difference", r"$T$ Difference"),
]

fig, axs = plt.subplots(
    nrows=1,
    ncols=len(pairs),
    figsize=(16, 6),
    constrained_layout=True,
)

for ax, (x, y) in zip(axs, pairs, strict=False):
    # Calculate Spearman correlation coefficient
    res = spearmanr(df[x], df[y])
    corr = res.statistic
    p = res.pvalue
    print(f"Spearman correlation between {x} and {y}: {corr:.2f} (p-value: {p:.2e})")

    p = sns.scatterplot(
        data=df,
        x=x,
        y=y,
        hue="Kakarla",
        style="Side",
        ax=ax,
    )
    p.set_title(f"{y} vs {x}")
    p.set_xlabel(x)
    p.set_ylabel(y)

In [ ]:
selected_columns = ["N Error", "LPA Error", "RPA Error", r"$K$ Difference", r"$T$ Difference"]

correlations = df[selected_columns].corr(method="spearman")
p_values = df[selected_columns].corr(method=lambda x, y: spearmanr(x, y).pvalue)
mask = np.triu(np.ones_like(correlations, dtype=bool))

sns.set_style("white")
sns.set_context("notebook", font_scale=1.2)

fig, axs = plt.subplots(1, 3, figsize=(16, 6))
sns.heatmap(
    correlations,
    # mask=mask,
    annot=False,
    fmt=".2f",
    cmap="coolwarm",
    cbar_kws={"label": "Spearman correlation coefficient"},
    vmin=-1,
    vmax=1,
    center=0,
    ax=axs[0],
)

for i in range(correlations.shape[0]):
    for j in range(correlations.shape[1]):
        # if i >= j:
        #     continue

        corr = f"{correlations.iloc[i, j]:.2f}"
        if correlations.iloc[i, j] > 0:
            corr = f"+{corr}"
        else:
            corr = f"{corr}"

        if p_values.iloc[i, j] < 0.001:
            p_val = r"$^{*}$"
        elif p_values.iloc[i, j] < 0.01:
            p_val = r"$^{**}$"
        elif p_values.iloc[i, j] < 0.05:
            p_val = r"$^{***}$"
        else:
            p_val = ""

        color = "black" if abs(correlations.iloc[i, j]) < 0.5 else "white"

        axs[0].text(
            i + 0.5,
            j + 0.5,
            f"{corr}{p_val}",
            ha="center",
            va="center",
            color=color,
            fontsize=12,
        )

axs[0].set_title("Correlation Matrix")

sns.boxenplot(
    data=df,
    x="Side",
    y=r"$K$ Difference",
    hue="Side",
    ax=axs[1],
    order=["Left", "Right"],
)
axs[1].set_title(r"$K$ Difference by Side")
axs[1].set_xlabel("")
axs[1].set_ylabel(r"Difference (mm)")

sns.boxenplot(
    data=df,
    x="Side",
    y=r"$T$ Difference",
    hue="Side",
    ax=axs[2],
    order=["Left", "Right"],
)
axs[2].set_title(r"$T$ Difference by Side")
axs[2].set_xlabel("")
axs[2].set_ylabel(r"Difference (mm)")

plt.tight_layout()
plt.show()

In [ ]:
# mean and std of the errors
for col in selected_columns:
    mean = df[col].mean()
    std = df[col].std()
    median = df[col].median()
    low_ci = df[col].quantile(0.025)
    high_ci = df[col].quantile(0.975)
    print(
        f"{col}: {mean:.2f} ± {std:.2f} (median: {median:.2f}, 95% CI: [{low_ci:.2f}, {high_ci:.2f}])"
    )

In [ ]:
df.to_csv(Path(r"./stability_results.csv"), index=False)